# How Old is This House (Manhattan)?

This is a tiny notebook inspired by the projects [How old is this house](https://how-old-is-this.house) and [Morphocode](https://io.morphocode.com/urban-layers).

Data is provided by [Data.gov](https://catalog.data.gov/dataset/building-footprints-92723) and [OSM](https://www.openstreetmap.org).

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkt

from lets_plot import *
LetsPlot.setup_html()

In [2]:
buildings_df = pd.read_csv('https://raw.githubusercontent.com'
                           '/HIL-HK/lets-plot-examples/master/data/manhattan_buildings.csv')
buildings_df = buildings_df.rename(columns={'CNSTRCT_YR': 'year', 'the_geom': 'geometry'})[['year', 'geometry']]
buildings_df.geometry = buildings_df.geometry.apply(wkt.loads)
buildings_df.year = np.where((buildings_df.year > 0)&(buildings_df.year <= 2020), buildings_df.year, 0)
buildings_gdf = gpd.GeoDataFrame(buildings_df, geometry='geometry')

boundary_gdf = gpd.read_file('https://raw.githubusercontent.com'
                             '/HIL-HK/lets-plot-examples/master/data/manhattan_boundary.geojson')

In [3]:
bc_agg_gdf = gpd.GeoDataFrame(
    buildings_gdf.groupby('year').geometry.agg(lambda s: s.unary_union).to_frame(name='geometry').reset_index(),
    geometry='geometry'
)
bc_agg_gdf.year = np.where(bc_agg_gdf.year == 0, np.nan, bc_agg_gdf.year)

In [4]:
xmin, ymin, xmax, ymax = boundary_gdf.total_bounds
year_breaks = [1750, 1800, 1850, 1900, 1950, 2000]
year_labels = [str(Y) for Y in year_breaks]

ggplot() + \
    geom_rect(xmin=xmin-.01, ymin=ymin-.01, xmax=xmax+.01, ymax=ymax+.01, fill='#252525', size=0) + \
    geom_polygon(data=boundary_gdf, color='#636363', fill='black', size=.5) + \
    geom_polygon(aes(fill='year'), data=bc_agg_gdf, \
                 tooltips=layer_tooltips().format('@year', 'd').line('Construction year|@year')) + \
    scale_fill_gradient(name='Construction year', low='#ca0020', high='#0571b0', \
                        breaks=year_breaks, labels=year_labels) + \
    ggsize(800, 600) + \
    ggtitle('Manhattan Buildings') + \
    theme(axis_title='blank', axis_text='blank', axis_ticks='blank', axis_line='blank')